In [424]:
import tabula
import pandas as pd
import os
import numpy as np
import re

In [3]:
filenames = ['best_sector_times_64.pdf','official_starting_grid_41.pdf',
'fastest_laps_24.pdf',
'history_chart_25.pdf','pit_stop_summary_29.pdf',
'lap_analysis_25.pdf','provisional_starting_grid_37.pdf',
'lap_chart_31.pdf','speed_trap_45.pdf',
'maximum_speeds_62.pdf']

In [8]:
for pidieffe in filenames:
    #print(pidieffe)
    frame_name = pidieffe[:-4]+'_df'
    #print(frame_name)
    exec( frame_name+" = tabula.read_pdf(pidieffe)")
    #exec(frame_name+".head()")

In [154]:
dfs = []
for _pdf in os.listdir('2016race0'):
    if (_pdf.lower().endswith(('.pdf'))) & ('jpg' not in _pdf.lower()):
        _name = _pdf[7:-9]+'_df'
#         print(_pdf)
#         print(_name)
        dfs.append(_name)
        exec( _name+" = tabula.read_pdf('2016race0/'+_pdf,pages='all')")
        exec(_name+".to_csv('2016race0/'+_name+'.csv')")

In [606]:
for _csv in os.listdir('2016race0'):
    if _csv.lower().endswith(('.csv')):
        _name = _csv[:-4]
#         print(_csv)
#         print(_name)
        exec(_name+" = pd.read_csv('2016race0/'+_name+'.csv')")

In [607]:
df = history_chart_2_df

In [608]:
df.rename(columns = {'NO':'NO.0','GAP':'GAP.0','TIME':'TIME.0'},inplace=True)

In [609]:
stopperz = df.loc[df['NO.0']=='NO'].index

In [610]:
last_stopper = -1
for _id,_val in enumerate(stopperz):
    #print (_id,_val)
    exec('df_'+str(_id)+'= df.loc[last_stopper+1:_val-1]' )
    last_stopper = _val

In [611]:
for j in range(len(stopperz)):
    for i in [0,1,2,3,4]:        
        exec('df_lap_'+str(j*5+(i+1))+' = df_'+str(j)+'[["NO.'+str(i)+'","GAP.'+str(i)+'","TIME.'+str(i)+'"]]')
        exec('df_lap_'+str(j*5+(i+1))+'["lap"] = '+str(j*5+(i+1)))
        exec('df_lap_'+str(j*5+(i+1))+'.columns = ["NO","GAP","TIME","LAP"]')

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [612]:
starters = df_lap_1['NO'].unique()

In [613]:
def split_PIT(x):
    if x['NO'] not in starters:
        if pd.notna(x['NO']):
            if x['NO'][:-4] in starters:
#            return [1,2,3,4]
                #print('correcting it')
                return pd.Series([x['NO'][:-4],'PIT',x['TIME'],x['LAP']])
#             else:
#                 #print('trashing it')
#                 return pd.Series([None, None, None, None])
        else:
            #print('nan')
            return pd.Series([None, None, None, None])
    else:
        #print('starter')
        return x
        #return [x['NO'],x['GAP'],x['TIME'],x['LAP']]

In [614]:
df_lap_18 = df_lap_18.apply(split_PIT, axis =1)

In [615]:
df_lap_18.columns = ["NO","GAP","TIME","LAP"]

In [616]:
def split_TIMES(x):
    if x['NO'] not in starters:
        return pd.Series([None, None, None, None])
    elif ' ' in x['GAP']:
        timez = x['GAP'].split()
        return pd.Series([x['NO'],timez[0],timez[1],x['LAP']])
    elif pd.isna(x['TIME']):
        return pd.Series([x['NO'],0,x['GAP'],x['LAP']])
    else:
        return x

In [617]:
df_lap_19 = df_lap_19.apply(split_TIMES,axis =1)

In [619]:
laps_dfs=[]
for j in range(len(stopperz)):
    for i in [0,1,2,3,4]:
        exec('laps_dfs.append(df_lap_'+str(j*5+(i+1))+')')

In [620]:
race_df = pd.concat(laps_dfs,ignore_index=True)

In [621]:
race_df.drop(race_df.index[race_df['NO'].isna()],axis=0, inplace=True)

In [622]:
race_df['GAP'].fillna(0,inplace=True)

In [623]:
pit_stop_df = pit_stop_summary_6_df

In [624]:
pit_stop_df = pit_stop_df[pit_stop_df['NO']!='NO']

In [625]:
pit_stop_df.rename(columns = {'S  T  OP':'STOP'},inplace=True)

/Users/guido/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [626]:
pit_stop_df['NO'] = pit_stop_df['NO'].astype('int64')

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [629]:
pit_stop_df['LAP'] = pit_stop_df['LAP'].astype('int64')

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [630]:
race_df['NO'] = race_df['NO'].astype('int64')

In [631]:
race_df['LAP'] = race_df['LAP'].astype('int64')

In [632]:
race_df.set_index(['NO','LAP'], inplace=True)
pit_stop_df.set_index(['NO','LAP'], inplace=True)

In [634]:
new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')

In [635]:
new_df.drop(columns = ['Unnamed: 0','Unnamed: 6'], inplace=True)

In [636]:
new_df

,,GAP,TIME,DRIVER,ENTRANT,TIME OF DAY,STOP,DURATION,TOTAL TIME
NO,LAP,,,,,,,,
5,1,0,1:36.916,NaN,NaN,NaN,NaN,NaN,NaN
7,1,1.514,1:38.430,NaN,NaN,NaN,NaN,NaN,NaN
6,1,2.400,1:39.316,NaN,NaN,NaN,NaN,NaN,NaN
33,1,3.158,1:40.074,NaN,NaN,NaN,NaN,NaN,NaN
19,1,4.108,1:41.024,NaN,NaN,NaN,NaN,NaN,NaN
44,1,4.373,1:41.289,NaN,NaN,NaN,NaN,NaN,NaN
55,1,5.149,1:42.065,NaN,NaN,NaN,NaN,NaN,NaN
27,1,6.518,1:43.434,NaN,NaN,NaN,NaN,NaN,NaN
3,1,6.860,1:43.776,NaN,NaN,NaN,NaN,NaN,NaN
